<a href="https://colab.research.google.com/github/amadords/Projetos-Publicos/blob/master/Classifica%C3%A7%C3%A3o_MultiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classificação MultiLabel
---

[![LinkedIn](https://img.shields.io/badge/LinkedIn-DanielSousaAmador-cyan.svg)](https://www.linkedin.com/in/daniel-sousa-amador)
[![GitHub](https://img.shields.io/badge/GitHub-amadords-darkblue.svg)](https://github.com/amadords)
[![Medium](https://img.shields.io/badge/Medium-DanielSousaAmador-white.svg)](https://daniel-s-amador.medium.com/)

![img](https://image.freepik.com/vetores-gratis/na-ilustracao-do-conceito-de-caminho_114360-1583.jpg)


    Observação: O presente notebook é material complementar para o artigo Classificação MultiLabel que você pode ler no link abaixo e que servirá de base teórica para tudo o que será feito abaixo.

[Classificação MultiLabel](link)

Em uma classificação, normalmente seguimos um caminho apenas. Treinamos algum algoritmo para que ele tente nos retornar a qual classe acha que determinado dado pertence. Além disso o que ele pode nos retornar é a probabilidade de ser de determinada classe ou não, porém sempre será retornada a classe com maior probabilidade como resultado da previsão.
Mas quando precisamos mudar esse caminho? Quando precisamos decidir além da **classificação binária** (Sobrevive ao desastre do Titanic ou Não, por exemplo) e da **Multi Classe** (Iris Versicolor, Virginica ou Setosa, por exemplo), o que fazer? Se houvesse a possibilidade de alguém sobreviver e não sobreviver ou de uma flor ser, por exemplo Versicolor e Virginica, ao mesmo tempo?
Talvez seja difícil pensar que esse tipo de cenário existe e caso exista, que a gente possa se deparar com ele, correto?! Mas vamos pensar no seguinte cenário: Em uma classificador de músicas, no **Spotify**, por exemplo, preciso saber se determinada música é, não apenas alegre, mas se ela também é relaxante, ou se determinada música é melancólica e depressiva etc. Então dentro desse cenário, parece fazer sentido pensar em classificar músicas em mais de um modo, correto? É isso que iremos fazer agora.

Note que usaremos apenas a Métrica de Avaliação **Hamming Loss** que utiliza a fórmula:

$$
\frac{Somatório\,dos\,Erros}{Nº\,de\,Labels}
$$

Quanto mais próximo de zero, melhor o resultado.

**Bibliotecas Necessárias**

In [1]:
!pip install scikit-multilearn -q

# para algoritmos adaptados usaremos KNN e Hierarquical ARAM NN
from skmultilearn.adapt import MLkNN, MLARAM
# para transformação de problemas usaremos Binary Relevance, Classifier Chain e Label Powerset
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
import pandas as pd

## Compreendendo os Dados

Veja que temos uma espécie de **Matriz Presença**, onde o 1 representa a presença e o 0 a não presença. Ou seja a primeira linha a música tem o valor 0 para `amazed-suprised`, `quiet-still`, `sad-lonely` e `angry-aggresive` porém tem o valor 1 para `happy-pleased` e `relaxing-clam`, o que significa que ela é uma música que traz felicidade e satisfação e também é relaxante, mas não pertence aos demais grupos.

Diferente de um problema onde temos para cada dado uma classe específica, aqui temos várias para cada uma, portanto não teremos apenas uma coluna para classe e sim as 6 primeiras.

In [2]:
df= pd.read_csv('Musica.csv', sep=",")
df.head()

,amazed-suprised,happy-pleased,relaxing-clam,quiet-still,sad-lonely,angry-aggresive,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,Mean_Acc1298_Mean_Mem40_MFCC_1,Mean_Acc1298_Mean_Mem40_MFCC_2,Mean_Acc1298_Mean_Mem40_MFCC_3,Mean_Acc1298_Mean_Mem40_MFCC_4,Mean_Acc1298_Mean_Mem40_MFCC_5,Mean_Acc1298_Mean_Mem40_MFCC_6,Mean_Acc1298_Mean_Mem40_MFCC_7,Mean_Acc1298_Mean_Mem40_MFCC_8,Mean_Acc1298_Mean_Mem40_MFCC_9,Mean_Acc1298_Mean_Mem40_MFCC_10,Mean_Acc1298_Mean_Mem40_MFCC_11,Mean_Acc1298_Mean_Mem40_MFCC_12,Mean_Acc1298_Std_Mem40_Centroid,Mean_Acc1298_Std_Mem40_Rolloff,Mean_Acc1298_Std_Mem40_Flux,Mean_Acc1298_Std_Mem40_MFCC_0,Mean_Acc1298_Std_Mem40_MFCC_1,Mean_Acc1298_Std_Mem40_MFCC_2,Mean_Acc1298_Std_Mem40_MFCC_3,Mean_Acc1298_Std_Mem40_MFCC_4,Mean_Acc1298_Std_Mem40_MFCC_5,Mean_Acc1298_Std_Mem40_MFCC_6,Mean_Acc1298_Std_Mem40_MFCC_7,Mean_Acc1298_Std_Mem40_MFCC_8,Mean_Acc1298_Std_Mem40_MFCC_9,Mean_Acc1298_Std_Mem40_MFCC_10,Mean_Acc1298_Std_Mem40_MFCC_11,Mean_Acc1298_Std_Mem40_MFCC_12,Std_Acc1298_Mean_Mem40_Centroid,Std_Acc1298_Mean_Mem40_Rolloff,Std_Acc1298_Mean_Mem40_Flux,Std_Acc1298_Mean_Mem40_MFCC_0,Std_Acc1298_Mean_Mem40_MFCC_1,Std_Acc1298_Mean_Mem40_MFCC_2,Std_Acc1298_Mean_Mem40_MFCC_3,Std_Acc1298_Mean_Mem40_MFCC_4,Std_Acc1298_Mean_Mem40_MFCC_5,Std_Acc1298_Mean_Mem40_MFCC_6,Std_Acc1298_Mean_Mem40_MFCC_7,Std_Acc1298_Mean_Mem40_MFCC_8,Std_Acc1298_Mean_Mem40_MFCC_9,Std_Acc1298_Mean_Mem40_MFCC_10,Std_Acc1298_Mean_Mem40_MFCC_11,Std_Acc1298_Mean_Mem40_MFCC_12,Std_Acc1298_Std_Mem40_Centroid,Std_Acc1298_Std_Mem40_Rolloff,Std_Acc1298_Std_Mem40_Flux,Std_Acc1298_Std_Mem40_MFCC_0,Std_Acc1298_Std_Mem40_MFCC_1,Std_Acc1298_Std_Mem40_MFCC_2,Std_Acc1298_Std_Mem40_MFCC_3,Std_Acc1298_Std_Mem40_MFCC_4,Std_Acc1298_Std_Mem40_MFCC_5,Std_Acc1298_Std_Mem40_MFCC_6,Std_Acc1298_Std_Mem40_MFCC_7,Std_Acc1298_Std_Mem40_MFCC_8,Std_Acc1298_Std_Mem40_MFCC_9,Std_Acc1298_Std_Mem40_MFCC_10,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BHSUM1,BHSUM2,BHSUM3
0,0,1,1,0,0,0,0.132498,0.077848,0.229227,0.602629,0.512861,0.467404,0.529733,0.573498,0.592831,0.520031,0.598853,0.537699,0.780658,0.462982,0.407108,0.684364,0.135824,0.245631,0.157515,0.301285,0.350107,0.459476,0.583274,0.430053,0.416198,0.581916,0.342758,0.309345,0.388929,0.323521,0.455207,0.261390,0.027559,0.149077,0.195433,0.571354,0.326404,0.246745,0.524645,0.354798,0.240244,0.239788,0.128689,0.173252,0.204863,0.131632,0.245653,0.144607,0.258203,0.470051,0.259909,0.613640,0.458314,0.434716,0.448941,0.370609,0.285647,0.663082,0.297080,0.273671,0.286411,0.197026,0.196244,0.164323,0.030017,0.253968,0.008473,0.240602,0.136735,0.058442,0.107594
1,1,0,0,0,0,1,0.384281,0.355249,0.167190,0.853089,0.260577,0.332757,0.153930,0.519381,0.268043,0.251955,0.459922,0.430814,0.654323,0.641021,0.356511,0.647367,0.367659,0.539078,0.100569,0.133502,0.337194,0.319752,0.349012,0.171182,0.191357,0.390569,0.289253,0.208641,0.341328,0.265669,0.273736,0.181791,0.028513,0.252827,0.258190,0.011351,0.236247,0.069285,0.192754,0.154258,0.128671,0.116726,0.059704,0.073697,0.080341,0.062701,0.075672,0.041256,0.207782,0.300735,0.888274,0.444000,0.294673,0.210429,0.132036,0.167474,0.205996,0.155514,0.086631,0.071462,0.067492,0.093526,0.085649,0.025101,0.182955,0.285714,0.156764,0.270677,0.191377,0.153728,0.197951
2,0,1,0,0,0,1,0.541782,0.356491,0.152246,0.791142,0.228276,0.471278,0.378166,0.559905,0.279949,0.555830,0.521424,0.477018,0.541562,0.383692,0.301562,0.209283,0.469560,0.435643,0.117424,0.233075,0.214409,0.270485,0.448359,0.446847,0.272785,0.626050,0.457692,0.257535,0.486894,0.433737,0.429170,0.250437,0.173474,0.240775,0.166080,0.134165,0.149816,0.312213,0.351309,0.550984,0.241480,0.351404,0.324881,0.102732,0.214608,0.182897,0.131488,0.092507,0.539964,0.291858,0.894329,0.316151,0.345686,0.218272,0.254165,0.392987,0.243410,0.529933,0.272590,0.153845,0.314687,0.198082,0.108067,0.140574,0.099

**Tamanho da base de dados**

592 linhas x 77 colunas

In [3]:
df.shape

(592, 77)

## Treinamento

**Separação da classe e previsores**


In [4]:
classe = df.iloc[:,0:6].values
previsores = df.iloc[:,7:78].values
classe

array([[0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1],
       ...,
       [0, 0, 1, 1, 1, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

**Divisão de treino e teste**

Lembre que o `random_state` garante que a "aleatoriedade" na divisão seja sempre a mesma. Isso significa que sempre que for rodada a divisão ela será a mesma, garantindo que o resultado não seja diferente sempre que for necessária a reexecução. 12 foi o número que escolhi, mas pode ser qualquer valor que você queira.

In [5]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,classe,test_size = 0.3,
                                                                  random_state = 12)

## Algoritmo Adaptado

###KNN

In [6]:
knn = MLkNN(k=3) 
knn.fit(X_treinamento, y_treinamento) 

MLkNN(ignore_first_neighbours=0, k=3, s=1.0)

**Previsão com dados de teste**



In [7]:
previsto = knn.predict(X_teste) 

# atenção na saída e motivo de não ser visualizada
previsto

<178x6 sparse matrix of type '<class 'numpy.int64'>'
	with 314 stored elements in List of Lists format>

**Hamming para avaliar preformance**


In [8]:
print(hamming_loss(y_teste, previsto)) 

0.20880149812734083


### Hierarquical ARAM NN

In [9]:
aram = MLARAM()
aram.fit(X_treinamento, y_treinamento) 

MLARAM(neurons=[<skmultilearn.adapt.mlaram.Neuron object at 0x7f694619beb8>,
                <skmultilearn.adapt.mlaram.Neuron object at 0x7f6946...
                <skmultilearn.adapt.mlaram.Neuron object at 0x7f69460ec668>, ...],
       threshold=0.02, vigilance=0.9)

**Previsão com dados de teste**

In [10]:
previsto = aram.predict(X_teste) 
previsto

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1.],
       ...,
       [0., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0.],
       [0., 1., 1., 0., 0., 0.]])

**Hamming para avaliar preformance**


In [11]:
print(hamming_loss(y_teste, previsto)) 

0.21910112359550563


## Transformação de Problema

### Binary Relevance

In [12]:
binary = BinaryRelevance(classifier = SVC())
binary.fit(X_treinamento, y_treinamento)
previsao = binary.predict(X_teste)
print(hamming_loss(y_teste, previsao))

0.21722846441947566


### Classifier Chain

In [13]:
chain = ClassifierChain(classifier = SVC())
chain.fit(X_treinamento, y_treinamento)
previsoes = chain.predict(X_teste)
print(hamming_loss(y_teste,previsoes))

0.23314606741573032


### Label PowerSet

In [14]:
label = LabelPowerset(classifier = SVC())
label.fit(X_treinamento, y_treinamento)
previsoes = label.predict(X_teste)
print(hamming_loss(y_teste,previsoes))

0.19288389513108614


# Obrigado!

Obrigado por ter disponibilizado um pouco do seu tempo e atenção aqui. Espero que, de alguma forma, tenha sido útil para seu crescimento. Se houver qualquer dúvida ou sugestão, não hesite em entrar em contato no [LinkedIn](https://www.linkedin.com/in/daniel-sousa-amador) e verificar meus outros projetos no [GitHub](https://github.com/amadords).

[![LinkedIn](https://img.shields.io/badge/LinkedIn-DanielSousaAmador-cyan.svg)](https://www.linkedin.com/in/daniel-sousa-amador)
[![GitHub](https://img.shields.io/badge/GitHub-amadords-darkblue.svg)](https://github.com/amadords)
[![Medium](https://img.shields.io/badge/Medium-DanielSousaAmador-white.svg)](https://daniel-s-amador.medium.com/)



<center><img width="90%" src="https://raw.githubusercontent.com/danielamador12/Portfolio/master/github.png"></center>